## Circle Group

Before we start building circle FFT, let's first build the circle group, which will be used as the domain of circle fft.

We choose a basic field GF(31), which has a order of 31. And then we extend it to a 2-degree extension field GF($31^2$), to get a multiplicative subgroup of order 32.

In [1]:
F31 = GF(31)
R31.<X> = F31[]
R31

Univariate Polynomial Ring in X over Finite Field of size 31

Now we extend F31 to a 2-degree extension field GF($31^2$).

In [2]:
C31.<i> = F31.extension(X^2 + 1)
C31

Finite Field in i of size 31^2

We want to find a group element of order 32, so we take the generator of GF($31^2$) and get 30th power of it.

**Fact**: For any prime field $\mathbb{F}_p$, where $p=2^n - 1$, the 2-degree extension over $\mathbb{F}_p$ is 
$K=\mathbb{F}_{p^2}$, then $K$ has a multiplicative subgroup with order $2^n$.

Proof:

$$
p^2 - 1 = (2^n - 1) * (2^n - 1) - 1 = 2^{2n} - 2^{n+1} = 2^n(2^n - 2)
$$

In our case, F31.order() - 1 is $2^n - 2$.

In [3]:

g = C31.multiplicative_generator()^(F31.order() - 1)
g

11*i + 29

Define a function to calculate the order of an element in the circle group.

In [4]:
def group_ord(elem):
    K = elem.parent()

    for i in range(1, K.order()):
        # An order of an element is defined by how much power we need to raise the element to get the identity element
        if elem**i == K.one():
            return i
    return "False"

Check that g is of order 32.

In [5]:
group_ord(g)

32

Define the group multiplication, which is similar to complex number multiplication.

In [6]:
def group_mul(g1, g2):
    x1, y1 = g1
    x2, y2 = g2
    return (x1 * x2 - y1 * y2) + (x1 * y2 + x2 * y1) * i 

Define the group inverse, which is also the J mapping in the paper.

In [7]:
def group_inv(g1):
    x1, y1 = g1
    return (x1 - y1 * i)

Define show function to print group.

In [8]:
def show(GG): 
    for j in range(0, len(GG)):
        t = GG[j]
        t0 = t[0]
        t1 = t[1]
        if t1 > F31(15) and t1 <= F31(30):
            t1_str = "-" + str(31-t1)
        else:
            t1_str = str(t1)
        print("i={}, log={}, ord={}, ({},{})".format(j, log_gen(g, t), group_ord(t), t0, t1_str))

Define log_gen function to calculate the power of an element in the circle group.

In [9]:
def log_gen(gen, t):
    K = gen.parent()
    for i in range(1, K.order()):
        if gen^i == t:
            return i
    return "Fail"

## Jump over the circle

Here we define the generator for `Circle Group` as chosen by Vitalik https://vitalik.eth.limo/general/2024/07/23/circlestarks.html

In [10]:
# optional
g = 10 + 5*i
group_ord(g)

32

Then we generate G5, which is a normal multiplicative group of order $2^5$.

G5's elements must be on the circle $x^2 + y^2 = 1$, due to:

$$
a^{p + 1} = 1, a = x + y * i
$$

$$  
a * a ^ p = (x + y * i) * (x - y * i) = x^2 + y^2 = 1
$$

Since $a^p$ is a Frobenius map, $a^p$ equals to the conjugate of $a$, which is $x - y * i$.

In [11]:
G5 = [g^i for i in range(0, 2^5)]
# check that G5's elements are on the circle x^2 + y^2 = 1
for t in G5:
    x, y = t
    assert x^2 + y^2 == 1
G5

[1,
 5*i + 10,
 7*i + 13,
 11*i + 2,
 27*i + 27,
 2*i + 11,
 13*i + 7,
 10*i + 5,
 i,
 10*i + 26,
 13*i + 24,
 2*i + 20,
 27*i + 4,
 11*i + 29,
 7*i + 18,
 5*i + 21,
 30,
 26*i + 21,
 24*i + 18,
 20*i + 29,
 4*i + 4,
 29*i + 20,
 18*i + 24,
 21*i + 26,
 30*i,
 21*i + 5,
 18*i + 7,
 29*i + 11,
 4*i + 27,
 20*i + 2,
 24*i + 13,
 26*i + 10]

Print information of G5's elements.

In [12]:
for j in range(0, 32):
    t = g^j
    print("i={}, log={}, ord={}, ({},{})".format(j, log_gen(g, t), group_ord(t), t[0], t[1]))

i=0, log=32, ord=1, (1,0)
i=1, log=1, ord=32, (10,5)
i=2, log=2, ord=16, (13,7)
i=3, log=3, ord=32, (2,11)
i=4, log=4, ord=8, (27,27)
i=5, log=5, ord=32, (11,2)
i=6, log=6, ord=16, (7,13)
i=7, log=7, ord=32, (5,10)
i=8, log=8, ord=4, (0,1)
i=9, log=9, ord=32, (26,10)
i=10, log=10, ord=16, (24,13)
i=11, log=11, ord=32, (20,2)
i=12, log=12, ord=8, (4,27)
i=13, log=13, ord=32, (29,11)
i=14, log=14, ord=16, (18,7)
i=15, log=15, ord=32, (21,5)
i=16, log=16, ord=2, (30,0)
i=17, log=17, ord=32, (21,26)
i=18, log=18, ord=16, (18,24)
i=19, log=19, ord=32, (29,20)
i=20, log=20, ord=8, (4,4)
i=21, log=21, ord=32, (20,29)
i=22, log=22, ord=16, (24,18)
i=23, log=23, ord=32, (26,21)
i=24, log=24, ord=4, (0,30)
i=25, log=25, ord=32, (5,21)
i=26, log=26, ord=16, (7,18)
i=27, log=27, ord=32, (11,29)
i=28, log=28, ord=8, (27,4)
i=29, log=29, ord=32, (2,20)
i=30, log=30, ord=16, (13,24)
i=31, log=31, ord=32, (10,26)


Check elements' operation on G5.

In [13]:
assert G5[1] * G5[1] == G5[2]
assert G5[3] * G5[4] == G5[7]
assert group_mul(G5[1], G5[2]) == G5[3]
assert group_inv(G5[1]) ==  G5[31]
assert G5[1] * group_inv(G5[1]) == 1

Generate G4 from G5 by squaring each element.

In [14]:
G4 = [t^2 for t in G5[:16]]
G4

[1,
 7*i + 13,
 27*i + 27,
 13*i + 7,
 i,
 13*i + 24,
 27*i + 4,
 7*i + 18,
 30,
 24*i + 18,
 4*i + 4,
 18*i + 24,
 30*i,
 18*i + 7,
 4*i + 27,
 24*i + 13]

Generate G3 from G4 by squaring each element.

In [15]:
G3 = [t^2 for t in G4[:8]]
G3

[1, 27*i + 27, i, 27*i + 4, 30, 4*i + 4, 30*i, 4*i + 27]

## Standard Position Coset

Let $Q$ be the generator of $G_{n+1}$, $D$ is a `standard position coset` of size $2^n$

$$
D = Q\cdot G_n
$$

where $ord(Q)=2^{n+1}$.

And $D$ is also a `twin-coset` of size $2^n$ defined with $G_{n-1}$:

$$
D = Q\cdot G_{n-1} \uplus Q^{-1}\cdot G_{n-1}
$$

The rotation $Q^2$ is exactly the generator of $G_n$.

Here we construct a series of cosets, including standard position cosets and twin cosets, where we can randomly find a coset for circle fft.

In [16]:
# Define a coset or G4 from a rotation Q, ord(Q) = 32
#
# Let Q = g (generator of G, whose order is 32)

D_standard_coset_4 = [g * t for t in G4]; show(D_standard_coset_4)

i=0, log=1, ord=32, (10,5)
i=1, log=3, ord=32, (2,11)
i=2, log=5, ord=32, (11,2)
i=3, log=7, ord=32, (5,10)
i=4, log=9, ord=32, (26,10)
i=5, log=11, ord=32, (20,2)
i=6, log=13, ord=32, (29,11)
i=7, log=15, ord=32, (21,5)
i=8, log=17, ord=32, (21,-5)
i=9, log=19, ord=32, (29,-11)
i=10, log=21, ord=32, (20,-2)
i=11, log=23, ord=32, (26,-10)
i=12, log=25, ord=32, (5,-10)
i=13, log=27, ord=32, (11,-2)
i=14, log=29, ord=32, (2,-11)
i=15, log=31, ord=32, (10,-5)


In [17]:
# D_twin_coset_4 is equal to D_standard_coset_4 in the set-theoretic sense, with different order

D_twin_coset_4 = [g * t for t in G3] + [group_inv(g) * t for t in G3]; show(D_standard_coset_4)

# `log` specifies the order of an element in the circle

i=0, log=1, ord=32, (10,5)
i=1, log=3, ord=32, (2,11)
i=2, log=5, ord=32, (11,2)
i=3, log=7, ord=32, (5,10)
i=4, log=9, ord=32, (26,10)
i=5, log=11, ord=32, (20,2)
i=6, log=13, ord=32, (29,11)
i=7, log=15, ord=32, (21,5)
i=8, log=17, ord=32, (21,-5)
i=9, log=19, ord=32, (29,-11)
i=10, log=21, ord=32, (20,-2)
i=11, log=23, ord=32, (26,-10)
i=12, log=25, ord=32, (5,-10)
i=13, log=27, ord=32, (11,-2)
i=14, log=29, ord=32, (2,-11)
i=15, log=31, ord=32, (10,-5)


In [18]:
# This is also a twin-coset of size 16

D_twin_coset_4_1 = [G5[3] * t for t in G3] + [group_inv(G5[3]) * t for t in G3]; show(D_twin_coset_4_1)

i=0, log=3, ord=32, (2,11)
i=1, log=7, ord=32, (5,10)
i=2, log=11, ord=32, (20,2)
i=3, log=15, ord=32, (21,5)
i=4, log=19, ord=32, (29,-11)
i=5, log=23, ord=32, (26,-10)
i=6, log=27, ord=32, (11,-2)
i=7, log=31, ord=32, (10,-5)
i=8, log=29, ord=32, (2,-11)
i=9, log=1, ord=32, (10,5)
i=10, log=5, ord=32, (11,2)
i=11, log=9, ord=32, (26,10)
i=12, log=13, ord=32, (29,11)
i=13, log=17, ord=32, (21,-5)
i=14, log=21, ord=32, (20,-2)
i=15, log=25, ord=32, (5,-10)


In [19]:
# Again

D_twin_coset_4_2 = [G5[5] * t for t in G3] + [group_inv(G5[5]) * t for t in G3]; show(D_twin_coset_4_2)

i=0, log=5, ord=32, (11,2)
i=1, log=9, ord=32, (26,10)
i=2, log=13, ord=32, (29,11)
i=3, log=17, ord=32, (21,-5)
i=4, log=21, ord=32, (20,-2)
i=5, log=25, ord=32, (5,-10)
i=6, log=29, ord=32, (2,-11)
i=7, log=1, ord=32, (10,5)
i=8, log=27, ord=32, (11,-2)
i=9, log=31, ord=32, (10,-5)
i=10, log=3, ord=32, (2,11)
i=11, log=7, ord=32, (5,10)
i=12, log=11, ord=32, (20,2)
i=13, log=15, ord=32, (21,5)
i=14, log=19, ord=32, (29,-11)
i=15, log=23, ord=32, (26,-10)


In [20]:
# Fact:
#   g^2 is the generator of G4
#   g * G4 is the standard position coset

G4 == [(g^2)^i for i in range(0, 16)]

True

In [21]:
# Define a coset or G3 from a rotation Q, ord(Q) = 16
#
# Let Q = g^2 (g is the generator of G5, whose order is 32)

D_standard_coset_3 = [g^2 * t for t in G3]; show(D_standard_coset_3)

i=0, log=2, ord=16, (13,7)
i=1, log=6, ord=16, (7,13)
i=2, log=10, ord=16, (24,13)
i=3, log=14, ord=16, (18,7)
i=4, log=18, ord=16, (18,-7)
i=5, log=22, ord=16, (24,-13)
i=6, log=26, ord=16, (7,-13)
i=7, log=30, ord=16, (13,-7)


## Squaring map

**Lemma**: For any twin-coset $D$ of size $N=2^n$, the image of $\pi(D)$ must be a twin-coset of size $2^{n-1}$.

$$
D = Q\cdot G_{n-1} \uplus Q^{-1}\cdot G_{n-1}
$$

$$
\pi(D) = \pi(Q)\cdot G_{n-2} \uplus \pi(Q)^{-1}\cdot G_{n-2}
$$

**Fact**: If $D$ is a standard position coset, so is $\pi(D)$.

In [22]:
def sq(D):
    rs = []
    for t in D:
        if t^2 not in rs:
            # x' == 2 * x^2 - 1
            rs += [t^2]
    return rs

In [23]:
# This is exactly the same coset we generated from G4 before

D_standard_coset_3 = sq(D_standard_coset_4); show(D_standard_coset_3)

i=0, log=2, ord=16, (13,7)
i=1, log=6, ord=16, (7,13)
i=2, log=10, ord=16, (24,13)
i=3, log=14, ord=16, (18,7)
i=4, log=18, ord=16, (18,-7)
i=5, log=22, ord=16, (24,-13)
i=6, log=26, ord=16, (7,-13)
i=7, log=30, ord=16, (13,-7)


## Circle FFT

To apply circle fft, let's generate a random polynomial's evaluation based on D_twin_coset_4_1 (or any other twin coset you prefer).

In [24]:
f = {}
from random import randint

for t in D_twin_coset_4_1:
    f[t] = C31(randint(0, 100))

f

{11*i + 2: 29,
 10*i + 5: 30,
 2*i + 20: 10,
 5*i + 21: 5,
 20*i + 29: 8,
 21*i + 26: 21,
 29*i + 11: 16,
 26*i + 10: 18,
 20*i + 2: 30,
 5*i + 10: 2,
 2*i + 11: 23,
 10*i + 26: 5,
 11*i + 29: 22,
 26*i + 21: 9,
 29*i + 20: 28,
 21*i + 5: 16}

Define the `pi` mapping applying to point's x coordinate.


In [25]:
def pi(t):
    # x^2 - y^2 == 2 * x^2 - 1 (x^2 + y^2 = 1)
    return C31(2 * t^2 - 1)

In the first step of ifft, we try to eliminate y coordinate to simplifier our computations in later steps.

We extract y from the polynomial by dividing polynomial in to 2 parts, f0 and f1, which correspond to the equation f[t] == f0[x] + y * f1[x].

t is a group element, x and y are the x and y coordinate of t respectively.

So that f0 and f1 are only functions of x coordinate.

In [26]:
def ifft_first_step(f):
    f0 = {}
    f1 = {}
    for t in f:
        x, y = t

        f0[x] = (f[t] + f[group_inv(t)]) / C31(2)
        f1[x] = (f[t] - f[group_inv(t)]) / (C31(2) * y)

        # Check that f is divided into 2 parts correctly
        assert f[t] == f0[x] + y * f1[x]

        # print("{}: {} = {} + {} * {}".format(t, f[t], f0[x], f1[x], y))

    return f0, f1

In normal steps, we deal with the polynomial exactly the same as the classical polynomial ifft, deeming the polynomial as a univariate polynomial, except that x is applied by the `pi` mapping.

Basically, we divide the polynomial into 2 parts, $f_0$ and $f_1$, which correspond to the equation:

$$
f[x] = f_0[x] + x * f_1[x]
$$

Thus

$$
f_0[x] = \frac{f[x] + f[-x]}{2}
$$
$$
f_1[x] = \frac{f[x] - f[-x]}{2x}
$$


In [27]:
def ifft_normal_step(f):

    if len(f) == 1:
        res = []
        for x in f:
            res.append(f[x])
        return res

    f0 = {}
    f1 = {}

    for x in f:
        assert x != 0, "f should be on coset"
        f0[pi(x)] = (f[x] + f[-x]) / C31(2)
        f1[pi(x)] = (f[x] - f[-x]) / (C31(2) * x)

        # Check that f is divided into 2 parts correctly
        assert f[x] == f0[pi(x)] + x * f1[pi(x)]

    return ifft_normal_step(f0) + ifft_normal_step(f1)

Now we have ifft_first_step and ifft_normal_step, we can define the ifft function.

In [28]:
def ifft(f):
    f0, f1 = ifft_first_step(f)
    f0 = ifft_normal_step(f0)
    f1 = ifft_normal_step(f1)

    return f0 + f1

Apply ifft to our random polynomial and see what we get.

In [29]:
coeffs = ifft(f)
coeffs

[17, 30, 17, 3, 4, 15, 16, 20, 1, 6, 24, 2, 12, 20, 9, 30]

Now we try to define the fft function.

Before that, we define `pie_group` function to generate next domain of a domain, which is equivalent to `sq` function in the x dimension (you can check that by focusing on the x coordinate in `sq` operation).


In [30]:
def pie_group(D):
    D_new = []
    for x in D:
        x_new = pi(x)
        if x_new not in D_new:
            D_new.append(x_new)

    # Check that the new domain is exactly half size of the old domain
    assert len(D_new) * 2 == len(D), "len(D_new) * 2 != len(D), {} * 2 != {}, D_new={}, D={}".format(len(D_new), len(D), D_new, D)
    
    return D_new

Define the first step of fft, which contains the same logic as ifft_first_step.

The basis of coefficients is like:

$$
1, v_1(x), v_2(x), v_1(x)v_2(x), y, yv_1(x), yv_2(x), yv_1(x)v_2(x) \quad (N = 8) \tag{1}
$$

where

$$
v_1(x) = x, v_2(x) = 2x^2-1, v_3(x) = 2(2x^2-1)^2-1, ...
$$

So to divide the polynomial into 2 parts, we just need to divide the coefficients into 2 parts, it's basis are also divided into 2 parts as follows:

$$
1, v_1(x), v_2(x), v_1(x)v_2(x) \tag{2}
$$

$$
y, yv_1(x), yv_2(x), yv_1(x)v_2(x) \tag{3}
$$

Notice that basis (2) is bit reversed order compared to normal basis, and basis (3) is like basis (2) but multiplied by y.

To eliminate y in normal steps' computation, we just consider basis of the second part of coefficients (which is actually basis (3)) as same as basis (2), and supply the y at the end of fft.

First, we define the first step of fft.

In [31]:
def fft_first_step(f, D):
    # Check that the polynomial and the domain have the same length
    assert len(f) == len(D), "len(f) != len(D), {} != {}, f={}, D={}".format(len(f), len(D), f, D)

    # divide the polynomial into 2 parts
    len_f = len(f)
    f0 = f[:len_f//2]
    f1 = f[len_f//2:]

    # halve the domain by simply removing the y coordinate
    D_new = []
    for t in D:
        x, _ = t
        if x not in D_new:
            D_new.append(x)

    # Check that the new domain is exactly half size of the old domain
    assert len(D_new) * 2 == len(D), "len(D_new) * 2 != len(D), {} * 2 != {}, D_new={}, D={}".format(len(D_new), len(D), D_new, D)

    return f0, f1, D_new

In normal steps, we deal with the polynomial exactly the same as the classical polynomial fft, deeming the polynomial as a univariate polynomial, except that $x$ is applied by the `pi` mapping.

Divide the polynomial into 2 parts, $f_0$ and $f_1$, which correspond to the equation:

$$
f[x] = f_0[x] + x * f_1[x]
$$

Merge $f_0$ and $f_1$ using $x$ from the domain, which corresponds to the equation above too.

In [32]:
def fft_normal_step(f, D):

    if len(f) == 1:
        return {D[0]: f[0]}
    
    next_domain = pie_group(D)

    # Check that the new domain is exactly half size of the old domain
    assert len(next_domain) * 2 == len(D), "len(next_domain) * 2 != len(D), {} * 2 != {}, next_domain={}, D={}".format(len(next_domain), len(D), next_domain, D)
    # Check that the polynomial and the domain have the same length
    assert len(f) == len(D), "len(f) != len(D), {} != {}, f={}, D={}".format(len(f), len(D), f, D)

    f0 = fft_normal_step(f[:len(f)//2], next_domain)
    f1 = fft_normal_step(f[len(f)//2:], next_domain)

    f_new = {}
    for x in D:
        f_new[x] = f0[pi(x)] + f1[pi(x)] * x

    # Check that f is divided into 2 parts correctly
    for x in D:
        if x != 0:
            assert f0[pi(x)] == (f_new[x] + f_new[-x]) / C31(2), "f0[pi(x)] = {}".format(f0[pi(x)])
            assert f1[pi(x)] == (f_new[x] - f_new[-x]) / (C31(2) * x), "f1[pi(x)] = {}".format(f1[pi(x)])
        else:
            assert f0[pi(x)] == f_new[x], "f0[pi(x)] = {}".format(f0[pi(x)])

    # Check that the polynomial and the domain have the same length
    assert len(f) == len(f_new), "len(f) != len(f_new), {} != {}, f={}, f_new={}, D={}".format(len(f), len(f_new), f, f_new, D)

    # Check that ifft and fft are correct inverse operations
    assert ifft_normal_step(f_new) == f, "ifft(f_new) != f, {} != {}".format(ifft_normal_step(f_new), f)
        
    return f_new

We have fft_first_step and fft_normal_step, we can define the fft function.

In [33]:
def fft(f, D):

    # Check that the polynomial and the domain have the same length
    assert len(f) == len(D), "len(f) != len(D), {} != {}, f={}, D={}".format(len(f), len(D), f, D)

    D_copy = D[:]
    f0, f1, D = fft_first_step(f, D)

    # Check that the polynomial and the domain have the same length
    assert len(f0) == len(D), "len(f0) != len(D), {} != {}, f0={}, D={}".format(len(f0), len(D), f0, D)
    assert len(f1) == len(D), "len(f1) != len(D), {} != {}, f1={}, D={}".format(len(f1), len(D), f1, D)

    f0 = fft_normal_step(f0, D)
    f1 = fft_normal_step(f1, D)

    f = {}
    # supply y to the polynomial
    for t in D_copy:
        x, y = t
        f[t] = f0[x] + f1[x] * y

    return f


Check that ifft and fft are correct inverse operations.

In [34]:
f_prime = fft(coeffs, D_twin_coset_4_1)

for t, s in zip(f, f_prime):
    assert t == s

for t in f:
    print("f[{}]={}, f_prime[{}]={}".format(t, f[t], t, f_prime[t]))
    assert f[t] == f_prime[t], "f[{}] != f_prime[{}], {} != {}".format(t, t, f[t], f_prime[t])

f[11*i + 2]=29, f_prime[11*i + 2]=29
f[10*i + 5]=30, f_prime[10*i + 5]=30
f[2*i + 20]=10, f_prime[2*i + 20]=10
f[5*i + 21]=5, f_prime[5*i + 21]=5
f[20*i + 29]=8, f_prime[20*i + 29]=8
f[21*i + 26]=21, f_prime[21*i + 26]=21
f[29*i + 11]=16, f_prime[29*i + 11]=16
f[26*i + 10]=18, f_prime[26*i + 10]=18
f[20*i + 2]=30, f_prime[20*i + 2]=30
f[5*i + 10]=2, f_prime[5*i + 10]=2
f[2*i + 11]=23, f_prime[2*i + 11]=23
f[10*i + 26]=5, f_prime[10*i + 26]=5
f[11*i + 29]=22, f_prime[11*i + 29]=22
f[26*i + 21]=9, f_prime[26*i + 21]=9
f[29*i + 20]=28, f_prime[29*i + 20]=28
f[21*i + 5]=16, f_prime[21*i + 5]=16


## Circle FRI

Basis:

$$
1, v_1(x), v_2(x), v_1(x)v_2(x), y, yv_1(x), yv_2(x), yv_1(x)v_2(x) \quad (N = 8)
$$


In [35]:
# Inputs:
#   f is the polynomial to be folded
#   D is the domain of f
#   r is the random number for random linear combination
# Outputs:
#   The first return value is the folded polynomial
#   The second return value is the new domain
def fold(f, D, r, debug=False):
    assert len(f) == len(D), "len(f) != len(D), {} != {}, f={}, D={}".format(len(f), len(D), f, D)

    # divide
    N = len(f)
    # left is the first half of f, of x from 1 to g^(N/2)
    left = f[:N//2]
    # right is the second half of f, of x from g^(N-1) to g^(N/2), which corresponds to minus x in left
    right = f[:N//2-1:-1]
    assert len(left) == len(right), "len(left) != len(right), {} != {}, left={}, right={}".format(len(left), len(right), left, right)

    for i, x in enumerate(D[:N//2]):
        # f == f0 + x * f1
        f0 = (left[i] + right[i]) / C31(2)
        f1 = (left[i] - right[i]) / (C31(2) * x)
        # f[:N//2] stores the folded polynomial
        if debug: print(f"f[{i}] = {f[i]} = ({left[i]} + {right[i]})/2 + {r} * ({left[i]} - {right[i]})/(2 * {x})")
        f[i] = f0 + r * f1
        # if debug: print(f"{f[i]} = ({left[i]} + {right[i]})/2 + {r} * ({left[i]} - {right[i]})/(2 * {x})")
        # reuse f[N//2:] to store new domain
        f[N//2 + i] = 2 * x^2 - 1

    # return the folded polynomial and the new domain
    return f[:N//2], f[N//2:]

In [36]:
from utils import log_2
from merlin.merlin_transcript import MerlinTranscript
from merkle import MerkleTree

def fri(f, D, degree, query_num, transcript, debug=False):

    assert len(f) == len(D), "len(f) != len(D), {} != {}, f={}, D={}".format(len(f), len(D), f, D)
    assert isinstance(transcript, MerlinTranscript)

    if debug: print(f"f={f}, D={D}")

    # first tree
    first_tree = MerkleTree(f)
    first_oracle = f

    # first random number
    transcript.append_message(b'first_tree', bytes(first_tree.root, 'ascii'))
    r = int.from_bytes(transcript.challenge_bytes(b'r', int(4)), 'big')

    # first step (J mapping)
    # for f in natural order, we just divide f into 2 parts from the middle
    N = len(f)
    assert N % 2 == 0, "N must be even, N={}".format(N)

    left = f[:N//2]
    right = f[:N//2-1:-1]
    assert len(left) == len(right), "len(left) != len(right), {} != {}, left={}, right={}".format(len(left), len(right), left, right)
    f = [None for _ in range(N//2)]
    for i, (_, y) in enumerate(D[:N//2]):
        f0 = (left[i] + right[i]) / C31(2)
        f1 = (left[i] - right[i]) / (C31(2) * y)
        f[i] = f0 + r * f1
        if debug: print(f"f[{i}] = {f[i]} = ({left[i]} + {right[i]})/2 + {r} * ({left[i]} - {right[i]})/(2 * {y})")

    print(f"f={f}")

    D = [x for x, _ in D[:N//2]]

    # fold
    trees = [MerkleTree(f)]
    oracles = [f[:]]
    # log_2(degree) - 1 because we have already done the first step
    for i in range(log_2(degree) - 1):
        # random number
        transcript.append_message(b'tree', bytes(trees[-1].root, 'ascii'))

        r = int.from_bytes(transcript.challenge_bytes(b'r', int(4)), 'big')
        f, D = fold(f, D, r, debug)

        if debug: print(f"f={f}, D={D}")

        # merkle tree
        trees.append(MerkleTree(f))
        oracles.append(f[:])

    # query paths
    max_size = N
    get_q = lambda transcript: int.from_bytes(transcript.challenge_bytes(b'query', int(4)), 'big') % max_size
    queries = [get_q(transcript) for _ in range(query_num)]

    query_paths = []
    for q in queries:
        max_size = N
        cur_path = []
        indices = []
        x0 = q
        x1 = max_size - q - 1
        if x1 < x0:
            x0, x1 = x1, x0

        if debug: print(f"max_size={max_size}")
        if debug: print(f"q={q}")
        if debug: print(f"x0={x0}, x1={x1}")
        
        cur_path.append((first_oracle[x0], first_oracle[x1]))
        indices.append(x0)
        q = x0
        max_size >>= 1

        for oracle in oracles:
            x0 = q
            x1 = max_size - q - 1
            if x1 < x0:
                x0, x1 = x1, x0

            if debug: print(f"q={q}")
            if debug: print(f"x0={x0}, x1={x1}")
            
            cur_path.append((oracle[x0], oracle[x1]))
            indices.append(x0)
            q = x0
            max_size >>= 1

        query_paths.append((cur_path, indices))

    # merkle paths
    merkle_paths = []
    for _, indices in query_paths:
        cur_query_paths = []
        for i, idx in enumerate(indices):
            if i == 0:
                if debug: print(f"i = {i}, idx = {idx}, first_oracle[idx] = {first_oracle[idx]}")
                cur_query_paths.append(first_tree.get_authentication_path(idx))
            else:
                cur_tree = trees[i - 1]
                if debug: print(f"i = {i}, idx = {idx}, oracles[i-1][idx] = {oracles[i-1][idx]}")
                cur_query_paths.append(cur_tree.get_authentication_path(idx))
        merkle_paths.append(cur_query_paths)

    return {
        'query_paths': query_paths,
        'merkle_paths': merkle_paths,
        'first_tree': first_tree.root,
        'intermediate_trees': [tree.root for tree in trees],
        'degree_bound': degree,
        'final_value': f[0],
    }

In [37]:
from merkle import verify_decommitment

def defri(fri_proof, degree_bound, T, query_num, transcript, debug=False):

    assert degree_bound >= fri_proof['degree_bound']
    degree_bound = fri_proof['degree_bound']

    assert isinstance(transcript, MerlinTranscript)

    first_tree = fri_proof['first_tree']
    intermediate_trees = fri_proof['intermediate_trees']

    transcript.append_message(b'first_tree', bytes(first_tree, 'ascii'))
    r = int.from_bytes(transcript.challenge_bytes(b'r', int(4)), 'big')

    fold_challenges = [r]
    for i in range(log_2(int(degree_bound)) - 1):
        transcript.append_message(b'tree', bytes(intermediate_trees[i], 'ascii'))
        r = int.from_bytes(transcript.challenge_bytes(b'r', int(4)), 'big')
        fold_challenges.append(r)

    get_q = lambda transcript: int.from_bytes(transcript.challenge_bytes(b'query', int(4)), 'big') % degree_bound
    queries = [get_q(transcript) for _ in range(query_num)]

    for q, (cur_path, _), mps in zip(queries, fri_proof['query_paths'], fri_proof['merkle_paths']):
        num_vars = degree_bound
        for i, mp in enumerate(mps):

            assert num_vars > 0, "num_vars must be positive, num_vars={}".format(num_vars)

            x0 = q
            x1 = num_vars - q - 1
            if x1 < x0:
                x0, x1 = x1, x0
            
            if debug: print(f"num_vars={num_vars}")
            if debug: print(f"q={q}")
            if debug: print(f"x0={x0}, x1={x1}")

            code_left, code_right = F31(cur_path[i][0]), F31(cur_path[i][1])

            table = T[i]
            if debug: print(f"i = {i}, table={table}")
            if i != len(mps) - 1:
                alpha = fold_challenges[i]
                f_code_folded = cur_path[i + 1][0 if x0 < num_vars / 4 else 1]
                assert f_code_folded == (code_left + code_right)/2 + alpha * (code_left - code_right)/(2*table[x0]), f"{f_code_folded} != ({code_left} + {code_right})/2 + {alpha} * ({code_left} - {code_right})/(2*{table[x0]})"
            else:
                assert fri_proof['final_value'] == code_left, f"{fri_proof['final_value']} != {code_left}"

            if i == 0:
                assert verify_decommitment(x0, code_left, mp, fri_proof['first_tree']), f"verify_decommitment(x0={x0}, code_left={code_left}, mp={mp}, fri_proof['first_tree']={fri_proof['first_tree']})"
            else:
                assert verify_decommitment(x0, code_left, mp, fri_proof['intermediate_trees'][i - 1]), f"verify_decommitment(x0={x0}, code_left={code_left}, mp={mp}, fri_proof['intermediate_trees'][i - 1]={fri_proof['intermediate_trees'][i - 1]})"

            num_vars >>= 1
            q = x0

In [38]:
f = [randint(0, 30) for _ in range(len(D_standard_coset_4))]
query_num = 4
fri_proof = fri(f, D_standard_coset_4, len(D_standard_coset_4), query_num, MerlinTranscript(b'FRI'), debug=True)
print("---------------- fri ----------------")
T = []
domain = D_standard_coset_4[:]
for i in range(log_2(len(domain)) + 1):
    if i != 0:
        T.append([x for x, _ in domain])
        domain = sq(domain)[:len(domain)//2]
    else:
        T.append([y for _, y in domain])
        domain = domain[:len(domain)//2]

defri(fri_proof, len(D_standard_coset_4), T, query_num, MerlinTranscript(b'FRI'), debug=True)

f=[24, 1, 6, 10, 3, 8, 29, 27, 25, 23, 3, 3, 13, 10, 24, 8], D=[5*i + 10, 11*i + 2, 2*i + 11, 10*i + 5, 10*i + 26, 2*i + 20, 11*i + 29, 5*i + 21, 26*i + 21, 20*i + 29, 29*i + 20, 21*i + 26, 21*i + 5, 29*i + 11, 20*i + 2, 26*i + 10]
f[0] = 1 = (24 + 8)/2 + 2944556307 * (24 - 8)/(2 * 5)
f[1] = 26 = (1 + 24)/2 + 2944556307 * (1 - 24)/(2 * 11)
f[2] = 29 = (6 + 10)/2 + 2944556307 * (6 - 10)/(2 * 2)
f[3] = 10 = (10 + 13)/2 + 2944556307 * (10 - 13)/(2 * 10)
f[4] = 3 = (3 + 3)/2 + 2944556307 * (3 - 3)/(2 * 10)
f[5] = 18 = (8 + 3)/2 + 2944556307 * (8 - 3)/(2 * 2)
f[6] = 9 = (29 + 23)/2 + 2944556307 * (29 - 23)/(2 * 11)
f[7] = 28 = (27 + 25)/2 + 2944556307 * (27 - 25)/(2 * 5)
f=[1, 26, 29, 10, 3, 18, 9, 28]
f[0] = 1 = (1 + 28)/2 + 144160416 * (1 - 28)/(2 * 10)
f[1] = 26 = (26 + 9)/2 + 144160416 * (26 - 9)/(2 * 2)
f[2] = 29 = (29 + 18)/2 + 144160416 * (29 - 18)/(2 * 11)
f[3] = 10 = (10 + 3)/2 + 144160416 * (10 - 3)/(2 * 5)
f=[30, 2, 8, 22], D=[13, 7, 24, 18]
f[0] = 30 = (30 + 22)/2 + 3212835317 *